# Import and Loading Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import module, secret_keys
from model_list import models
import pandas as pd

hf_api_key             = secret_keys.HF_TOKEN                   #<insert your own huggingface token here>
openai_api_key         = secret_keys.OPENAI_API_KEY_TEAM        #<insert your own openai token here>

# Load Dataset

These three IDs are three-shot example IDs in each dataset - ['NCT00000620', 'NCT01483560', 'NCT04280783']. Do not run generation or evaluation on them.

In [3]:
data_pub = pd.read_csv('data_new/CT-Pub-With-Examples-Corrected.csv')
print(data_pub.shape)
data_pub.head(2)

(103, 12)


,NCTId,BriefTitle,EligibilityCriteria,BriefSummary,Conditions,Interventions,PrimaryOutcomes,TrialGroup,API_BaselineMeasures,API_BaselineMeasures_Corrected,Paper_BaselineMeasures,Paper_BaselineMeasures_Corrected
0,NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Inclusion Criteria:\n\n* Diagnosed with type 2...,The purpose of this study is to prevent major ...,"Atherosclerosis, Cardiovascular Diseases, Hype...","Anti-hyperglycemic Agents, Anti-hypertensive A...",First Occurrence of a Major Cardiovascular Eve...,hypertension,"Age, Continuous, Gender, Ethnicity (NIH/OMB), ...","`Age, Continuous`, `Gender`, `Ethnicity (NIH/O...","Age, Female sex, Median duration of diabetes, ...","`Age`, `Female sex`, `Median duration of diabe..."
1,NCT00126737,Home-Based Exercise and Weight Control Program...,Inclusion Criteria:\n\n* Male \& female 50 yea...,The purpose of this study is to determine whet...,"Chronic Diseases, Obesity, Osteoarthritis, Pain,","Weight Control Nutritional Program, Home-based...","WOMAC Function, Physical Scale SF-36v, Mental ...",obesity,"Age, Continuous, Sex: Female, Male, Race/Ethni...","`Age, Continuous`, `Sex: Female, Male`, `Race/...","Age, Duration of OA, Kellgren-Lawrence Classif...","`Age`, `Duration of OA`, `Kellgren-Lawrence Cl..."


In [6]:
data_repo = pd.read_csv('data_new/CT-Repo-With-Examples-Processed-Version-Corrected.csv')
print(data_repo.shape)
data_repo.head(2)

(1693, 10)


,NCTId,BriefTitle,EligibilityCriteria,BriefSummary,Conditions,Interventions,PrimaryOutcomes,TrialGroup,API_BaselineMeasures,API_BaselineMeasures_Corrected
0,NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Inclusion Criteria:\n\n* Diagnosed with type 2...,The purpose of this study is to prevent major ...,"Atherosclerosis, Cardiovascular Diseases, Hype...","Anti-hyperglycemic Agents, Anti-hypertensive A...",First Occurrence of a Major Cardiovascular Eve...,hypertension,"Age, Continuous, Gender, Ethnicity (NIH/OMB), ...","`Age, Continuous`, `Gender`, `Ethnicity (NIH/O..."
1,NCT00003901,Prognostic Study of Metastases in Patients Wit...,Inclusion Criteria:\n\n1. Patient must be ≥ 18...,RATIONALE: Prognostic testing for early signs ...,"Lung Cancer,","immunohistochemistry staining method, biopsy, ...",Overall Survival in Lymph Nodes Examined Patie...,cancer,"Age, Continuous, Gender, Race/Ethnicity, Custo...","`Age, Continuous`, `Gender`, `Race/Ethnicity, ..."


## Example of Generation using GPT-4o with 0-shot prompts (CT-Pub)

In [7]:
model_name   = models['gpt-4o']

row = next(data_pub.iterrows())[1]
system_message, question = module.build_zeroshot_prompt(data_pub, row)
model_query = module.system_user_template(system_message, question)
model_response = module.run_generation_single_openai(model_query, model_name, 
                                                openai_api_key, temperature=0.0)
print(system_message)
print(question)
print(model_response)


You are a helpful assistant with experience in the clinical domain and clinical trial design.     You'll be asked queries related to clinical trials. These inquiries will be delineated by a '##Question' heading.     Inside these queries, expect to find comprehensive details about the clinical trial structured within specific subsections,     indicated by '<>' tags. These subsections include essential information such as the trial's title, brief summary,     condition under study, inclusion and exclusion criteria, intervention, and outcomes.In answer to this question, return a list of probable baseline features (each feature should be enclosed within a pair of backticks     and each feature should be separated by commas from other features) of the clinical trial.     Baseline features are the set of baseline or demographic characteristics that are assessed at baseline and used in the analysis of the     primary outcome measure(s) to characterize the study population and assess validity.

## Example of Generation using llama3-70b-it with 3-shot prompts

In [9]:
model_hf_endpoint = models['llama3-70b-it']

row = next(data_pub.iterrows())[1]
system_message, question = module.build_three_shot_prompt(data_pub, row, ref_col_name='Paper_BaselineMeasures_Corrected')
model_query = module.system_user_template(system_message, question)
model_response = module.run_generation_single_hf_models(model_query, model_hf_endpoint, 
                                                hf_api_key, temperature=0.0)
print(system_message)
print(question)
print(model_response)


You are a helpful assistant with experience in the clinical domain and clinical trial design.     You'll be asked queries related to clinical trials. These inquiries will be delineated by a '##Question' heading.     Inside these queries, expect to find comprehensive details about the clinical trial structured within specific subsections,     indicated by '<>' tags. These subsections include essential information such as the trial's title, brief summary,     condition under study, inclusion and exclusion criteria, intervention, and outcomes.In answer to this question, return a list of probable baseline features (each feature should be enclosed within a pair of backticks     and each feature should be separated by commas from other features) of the clinical trial.     Baseline features are the set of baseline or demographic characteristics that are assessed at baseline and used in the analysis of the     primary outcome measure(s) to characterize the study population and assess validity.

# Example of Eval using GPT-4o

In [11]:
row = data_pub.iloc[0]
reference_list = row['Paper_BaselineMeasures_Corrected']

#<insert any candidate list generated by any LLM here>
candidate_list = "`Age`, `Sex`, `Race`, `Body Mass Index (BMI)`, `Duration of Diabetes`, `HbA1c`, `Blood Pressure`, `Lipid Levels`, \
    `History of Cardiovascular Disease (CVD)`, `Smoking Status`, `Medication Use`, `Fasting Plasma Glucose`, `2-hour Postload Glucose`, \
        `History of Coronary Revascularization`, `History of Peripheral or Carotid Revascularization`, `Angina Status`, `Subclinical Disease Indicators`, \
            `Number of CVD Risk Factors`"

qstart = module.get_question_from_row(row)
system_message, question = module.build_gpt4_eval_prompt(module.extract_elements_v2(reference_list), 
                                                         module.extract_elements_v2(candidate_list), 
                                                         qstart)
model_response = module.run_evaluation_with_gpt4o(system_message, question, openai_api_key)

print(system_message)
print(question)
print(model_response)


        You are an expert assistant in the medical domain and clinical trial design. You are provided with details of a clinical trial.
        Your task is to determine which candidate baseline features match any feature in a reference baseline feature list for that trial. 
        You need to consider the context and semantics while matching the features.

        For each candidate feature:

            1. Identify a matching reference feature based on similarity in context and semantics.
            2. Remember the matched pair.
            3. A reference feature can only be matched to one candidate feature and cannot be further considered for any consecutive matches.
            4. If there are multiple possible matches (i.e. one reference feature can be matched to multiple candidate features or vice versa), choose the most contextually similar one.
            5. Also keep track of which reference and candidate features remain unmatched.

        Once the matching is complete, p